In [ ]:
# jupyter notebook tested on docker tensorflow/tensorflow:2.4.1-gpu-jupyter

import os
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers.experimental import preprocessing
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization, Reshape, LeakyReLU, Activation 
from tensorflow.keras.layers import Conv2D, MaxPool2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam, SGD

from PIL import Image

os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

from tensorflow.keras.mixed_precision import experimental as mixed_precision
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION"] = '1'

%matplotlib inline

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        print('{} memory growth: {}'.format(device, tf.config.experimental.get_memory_growth(device)))
else:
    print("Not enough GPU hardware devices available")

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

In [ ]:
# Seeding
np.random.seed(0)
tf.random.set_seed(0)

# Parameters
IMG_SIZE = 368
BATCH_SIZE = 256
WORKERS = 36
Channel = 3  # color images

## Latent space
latent_dim = 1024

In [ ]:
### model ###

inputs = Input(shape=(IMG_SIZE, IMG_SIZE, Channel), name="inputs")
x = inputs

x = Conv2D(32, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(16, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(8, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(8, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
temp = MaxPool2D((2, 2))(x)

x = Flatten()(temp)

units = x.shape[1]
encoded = Dense(latent_dim, name="latent")(x)

x = Dense(units)(encoded)
x = LeakyReLU(alpha=0.2)(x)
x = Reshape((temp.shape[1], temp.shape[2], temp.shape[3]))(x)

x = Conv2DTranspose(8, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(8, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(16, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(32, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(Channel, (3, 3), strides=1, padding="same")(x)
x = Activation("sigmoid", name="outputs")(x)

outputs = x

In [ ]:
# AutoEncodr
autoencoder = Model(inputs, outputs)
autoencoder.summary()

In [ ]:
# Encoder
encoder = Model(inputs, encoded)
encoder.summary()

In [ ]:
# load weight
# set path to the trained weight

autoencoder.load_weights( #path to 'AutoEncoder_weight.h5')

In [ ]:
####################

In [ ]:
# set image_directory to encode
# assumed directory structure is 'YOUR_TEST_DIRECTORY/SUB_DIRECTORY/IMAGES'

test_dir = # YOUR_TEST_DIRECTORY
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
                                                              samplewise_center=False,
                                                              featurewise_std_normalization=False,
                                                              samplewise_std_normalization=False,
                                                              zca_whitening=False,
                                                              zca_epsilon=1e-6,
                                                              rotation_range=0,
                                                              width_shift_range=0,
                                                              height_shift_range=0,
                                                              brightness_range=None,
                                                              shear_range=0.0,
                                                              zoom_range=0.0,
                                                              channel_shift_range=0.0,
                                                              fill_mode='nearest',
                                                              cval=0.0,
                                                              horizontal_flip=False,
                                                              vertical_flip=False,
                                                              rescale=1./255,
                                                              preprocessing_function=None,
                                                              data_format=None,
                                                              validation_split=0.0,
                                                              dtype=None)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMG_SIZE, IMG_SIZE),
                                                 color_mode='rgb', shuffle=False, batch_size=BATCH_SIZE,
                                                 class_mode=None, interpolation='lanczos')

In [ ]:
test_filenames = test_generator.filepaths
test_filenames

In [ ]:
EncodedImages = encoder.predict(test_generator, workers=WORKERS, verbose=1)
EncodedImages

In [ ]:
print(EncodedImages.shape)